In [ ]:
import requests
import time
import pandas as pd
import os
import json

# ========================
# Funciones auxiliares
# ========================

def obtener_info_juego_completa(appid):
    """Consulta la API de Steam y extrae campos útiles para recomendación + interfaz."""
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    try:
        r = requests.get(url, timeout=10)
        data = r.json()
        juego_data = data.get(str(appid), {})
        if juego_data.get("success"):
            info = juego_data["data"]
            return {
                "appid": appid,
                "name": info.get("name"),
                "genres": [g["description"] for g in info.get("genres", [])],
                "developers": info.get("developers", []),
                "short_description": info.get("short_description", ""),
                "header_image": info.get("header_image", ""),
                "categories": [c["description"] for c in info.get("categories", [])],
                "release_date": info.get("release_date", {}).get("date", ""),
                "metacritic_score": info.get("metacritic", {}).get("score", None),
                "price": info.get("price_overview", {}).get("final_formatted", None),
            }
        else:
            print(f"❌ Juego {appid} no disponible en Steam (success: false).")
    except Exception as e:
        print(f"⚠️ Error con appid {appid}: {e}")
    return None


def guardar_resultados(resultados, archivo):
    if os.path.exists(archivo):
        existentes = pd.read_csv(archivo)
        nuevos = pd.DataFrame(resultados)
        combinados = pd.concat([existentes, nuevos], ignore_index=True)
        combinados = combinados.drop_duplicates(subset="appid")
    else:
        combinados = pd.DataFrame(resultados)
    combinados.to_csv(archivo, index=False)


def guardar_json_sin_duplicados(lista, archivo):
    existentes = []
    if os.path.exists(archivo):
        with open(archivo) as f:
            existentes = json.load(f)
    actualizados = list(set(existentes).union(set(lista)))
    with open(archivo, "w") as f:
        json.dump(actualizados, f)


def cargar_json(archivo):
    if os.path.exists(archivo):
        with open(archivo) as f:
            return json.load(f)
    return []

def cargar_ya_procesados(archivo):
    if os.path.exists(archivo):
        try:
            return pd.read_csv(archivo)["appid"].astype(str).tolist()
        except:
            return []
    return []

# ========================
# Proceso principal en bucle
# ========================

def ejecutar_descarga_completa(lista_ids, archivo_salida="steam_juegos_metadata.csv",
                                archivo_pendientes="pendientes_steam.json", max_intentos=5):

    procesados = set(cargar_ya_procesados(archivo_salida))
    pendientes = set(cargar_json(archivo_pendientes))
    objetivo = set(lista_ids)

    # Intentar hasta agotar intentos
    for intento in range(1, max_intentos + 1):
        print(f"\n🔁 RONDA {intento}/{max_intentos}")
        ids_a_hacer = list((objetivo - procesados) | pendientes)
        if not ids_a_hacer:
            print("✅ Todos los juegos han sido procesados correctamente.")
            break

        print(f"🎯 Juegos restantes: {len(ids_a_hacer)}")
        resultados = []
        fallidos = []

        for i, appid in enumerate(ids_a_hacer, 1):
            print(f"⏳ [{i}/{len(ids_a_hacer)}] Consultando appid: {appid}")
            info = obtener_info_juego_completa(appid)
            if info:
                resultados.append(info)
                procesados.add(str(appid))
            else:
                fallidos.append(str(appid))

            if i % 10 == 0 or i == len(ids_a_hacer):
                guardar_resultados(resultados, archivo_salida)
                guardar_json_sin_duplicados(fallidos, archivo_pendientes)
                resultados = []

            time.sleep(0.2)  # evitar rate limit

    print("🏁 Proceso finalizado tras completar todas las rondas.")


In [ ]:
import pandas as pd

# --------------------------
# Cargar item_id desde juegos.csv
# --------------------------
def cargar_ids_desde_csv(ruta_csv="juegos.csv"):
    df = pd.read_csv(ruta_csv)
    df["item_id"] = df["item_id"].astype(str)
    return df["item_id"].tolist()

# --------------------------
# Ejecutar proceso principal
# --------------------------
from pathlib import Path

# Asegura que funciones auxiliares están definidas
# (Ya las definimos en tu script anterior: obtener_info_juego_completa, guardar_resultados, etc.)

# Cargar item_ids desde juegos.csv
lista_ids = cargar_ids_desde_csv("juegos.csv")

# Ejecutar scraping completo con reintentos y guardado incremental
ejecutar_descarga_completa(
    lista_ids,
    archivo_salida="steam_juegos_metadata.csv",
    archivo_pendientes="pendientes_steam.json",
    max_intentos=5
)
